# 🏭 EXO_LAUNCHER_COLAB.ipynb - USINE AUTOMATISÉE EXODUS

**Système Nerveux Central EXODUS pour Google Colab**  
**Transforme Google Colab en usine automatisée de génération vidéo**

## 📋 PRÉREQUIS

- Compte Google (pour Colab + Drive)
- Google Drive monté (les assets seront stockés sur Drive pour persistance)
- Runtime GPU activé (recommandé pour Blender Cycles)

## 🎯 OBJECTIF

Pipeline complet automatisé : **SOURCING → INQUISITION → CORTEX → MANUFACTORUM**

---

**La Voie Royale est mon guide, les 300k sont mon objectif.**


## ÉTAPE 1 : MONTER GOOGLE DRIVE (DOCTRINE PERSISTANCE)

Montage automatique de Google Drive pour stocker les assets et résultats de manière persistante.


In [ ]:
# Montage Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Chemin vers le dossier EXODUS sur Drive
EXODUS_DRIVE_PATH = '/content/drive/MyDrive/EXODUS_SYSTEM'

# Vérifier si le dossier existe
import os
from pathlib import Path

if not os.path.exists(EXODUS_DRIVE_PATH):
    print(f"[INFO] Création du dossier EXODUS sur Drive : {EXODUS_DRIVE_PATH}")
    os.makedirs(EXODUS_DRIVE_PATH, exist_ok=True)
else:
    print(f"[OK] Dossier EXODUS trouvé sur Drive : {EXODUS_DRIVE_PATH}")

print(f"[INFO] Chemin Drive EXODUS : {EXODUS_DRIVE_PATH}")


## ÉTAPE 2 : CLONER LE REPOSITORY GITHUB

Clonage du repository EXODUS_SYSTEM depuis GitHub (si pas déjà présent sur Drive).


In [ ]:
# Cloner le repository GitHub (si pas déjà présent)
import subprocess
import shutil
from pathlib import Path

REPO_URL = "https://github.com/ghostainzooalgon/EXODUS_SYSTEM.git"
WORK_DIR = "/content/EXODUS_SYSTEM"

# Vérifier si le dossier EXODUS existe déjà sur Drive
if os.path.exists(EXODUS_DRIVE_PATH) and any(Path(EXODUS_DRIVE_PATH).iterdir()):
    print(f"[INFO] Repository déjà présent sur Drive. Utilisation de {EXODUS_DRIVE_PATH}")
    WORK_DIR = EXODUS_DRIVE_PATH
else:
    print(f"[INFO] Clonage du repository GitHub...")
    if os.path.exists(WORK_DIR):
        shutil.rmtree(WORK_DIR)
    
    result = subprocess.run(
        ["git", "clone", REPO_URL, WORK_DIR],
        capture_output=True,
        text=True
    )
    
    if result.returncode != 0:
        print(f"[ERROR] Erreur lors du clonage : {result.stderr}")
    else:
        print(f"[SUCCESS] Repository cloné : {WORK_DIR}")

# Changer de répertoire
os.chdir(WORK_DIR)
print(f"[INFO] Répertoire de travail : {os.getcwd()}")


## ÉTAPE 3 : INSTALLER LES DÉPENDANCES PYTHON

Installation des dépendances Python nécessaires pour les Segments 01 et 02.


In [ ]:
# Installation des dépendances Python
!pip install -q opencv-python opencv-contrib-python mediapipe openai-whisper numpy json5 tqdm pathlib2 psutil pydub

print("[OK] Dépendances Python installées")


## ÉTAPE 4 : INSTALLER BLENDER (POUR SEGMENT 03)

Téléchargement et installation de Blender sur Google Colab.
**Doctrine Persistance** : Vérifie si Blender est déjà présent sur Drive pour éviter le téléchargement répétitif.


In [ ]:
# Installation Blender pour Segment 03
import urllib.request
import zipfile
from pathlib import Path

BLENDER_VERSION = "4.0"
BLENDER_URL = f"https://download.blender.org/release/Blender{BLENDER_VERSION}/blender-{BLENDER_VERSION}.2-linux-x64.tar.xz"
BLENDER_DRIVE_PATH = Path(EXODUS_DRIVE_PATH) / "tools" / "blender"
BLENDER_LOCAL_PATH = Path("/content/blender")

# Doctrine Persistance : Vérifier si Blender existe déjà sur Drive
if BLENDER_DRIVE_PATH.exists() and (BLENDER_DRIVE_PATH / "blender").exists():
    print(f"[OK] Blender trouvé sur Drive : {BLENDER_DRIVE_PATH}")
    print(f"[INFO] Utilisation de Blender depuis Drive (pas de téléchargement)")
    BLENDER_EXE = BLENDER_DRIVE_PATH / "blender"
else:
    print(f"[INFO] Blender non trouvé sur Drive. Installation...")
    
    # Téléchargement Blender
    blender_tar = "/content/blender.tar.xz"
    print(f"[INFO] Téléchargement de Blender depuis {BLENDER_URL}...")
    urllib.request.urlretrieve(BLENDER_URL, blender_tar)
    
    # Extraction
    print(f"[INFO] Extraction de Blender...")
    import tarfile
    with tarfile.open(blender_tar, 'r:xz') as tar:
        tar.extractall("/content")
    
    # Trouver le binaire blender
    blender_exe = Path("/content").glob("blender-*/blender").__next__()
    
    # Copier vers Drive pour persistance
    BLENDER_DRIVE_PATH.parent.mkdir(parents=True, exist_ok=True)
    if not BLENDER_DRIVE_PATH.exists():
        shutil.copytree(blender_exe.parent, BLENDER_DRIVE_PATH)
        print(f"[OK] Blender copié sur Drive : {BLENDER_DRIVE_PATH}")
    
    BLENDER_EXE = BLENDER_DRIVE_PATH / "blender"
    
    # Nettoyer
    os.remove(blender_tar)
    
    print(f"[SUCCESS] Blender installé : {BLENDER_EXE}")

# Vérifier que blender est exécutable
import stat
os.chmod(str(BLENDER_EXE), stat.S_IRWXU | stat.S_IRGRP | stat.S_IXGRP | stat.S_IROTH | stat.S_IXOTH)

print(f"[OK] Blender prêt : {BLENDER_EXE}")


## ÉTAPE 5 : INSTALLER FFMPEG

Installation de FFmpeg pour la post-production vidéo (Segment 03).


In [ ]:
# Installation FFmpeg
!apt-get update -qq
!apt-get install -y -qq ffmpeg

# Vérifier l'installation
result = subprocess.run(["ffmpeg", "-version"], capture_output=True, text=True)
if result.returncode == 0:
    version_line = result.stdout.split('\n')[0]
    print(f"[OK] FFmpeg installé : {version_line[:60]}")
else:
    print("[ERROR] FFmpeg non trouvé")


## ÉTAPE 6 : VÉRIFICATION DE L'ENVIRONNEMENT (DRY-RUN)

Vérification complète de l'environnement avant l'exécution (Assets, Outils, Permissions, Syntaxe).


In [ ]:
# Vérification de l'environnement (protocole --dry-run)
!python EXO_PRIME_ORCHESTRATOR.py --dry-run


## ÉTAPE 7 : LANCER L'ORCHESTRATEUR (EXÉCUTION COMPLÈTE)

Lancement du pipeline complet EXODUS.
L'orchestrateur demandera :
- **Mode global** : DRAMA (avec audio) ou SILENT (sans audio)
- **Nombre de variantes** : N variantes par vidéo source

Placez vos vidéos sources dans `01_EYE_INQUISITION/Raw_Videos/` avant de lancer.


In [ ]:
# Lancer l'Orchestrateur Suprême
# Note: L'orchestrateur demandera interactivement le mode et le nombre de variantes
# Pour exécution non-interactive, modifiez EXO_PRIME_ORCHESTRATOR.py pour accepter des arguments

import sys
import os

# Ajouter le chemin Blender au PATH si nécessaire
if 'BLENDER_EXE' in locals():
    blender_bin_dir = str(Path(BLENDER_EXE).parent)
    if blender_bin_dir not in os.environ.get('PATH', ''):
        os.environ['PATH'] = f"{blender_bin_dir}:{os.environ.get('PATH', '')}"

print("=" * 80)
print("PRÊT POUR L'EXÉCUTION COMPLÈTE")
print("=" * 80)
print("[INFO] Pour lancer l'orchestrateur, exécutez la cellule suivante ou :")
print("       python EXO_PRIME_ORCHESTRATOR.py")
print("=" * 80)


## 🚀 EXÉCUTION COMPLÈTE (LANCEMENT FINAL)

**Exécuter cette cellule pour lancer le pipeline complet EXODUS.**

L'orchestrateur traitera automatiquement toutes les vidéos dans `Raw_Videos/` et générera les variantes dans `Exports_4K/`.


In [ ]:
# LANCEMENT FINAL DE L'ORCHESTRATEUR
# Décommentez et modifiez les paramètres selon vos besoins

# Mode : "DRAMA" ou "SILENT"
MODE = "DRAMA"  # Changez selon vos besoins

# Nombre de variantes par vidéo source
NUM_VARIANTES = 3  # Changez selon vos besoins

# Pour exécution interactive, utilisez simplement :
# !python EXO_PRIME_ORCHESTRATOR.py

# Pour exécution non-interactive, vous devrez modifier EXO_PRIME_ORCHESTRATOR.py
# pour accepter --mode et --num-variantes en arguments

print("=" * 80)
print(f"MODE: {MODE}")
print(f"VARIANTES: {NUM_VARIANTES}")
print("=" * 80)
print("[INFO] Pour lancer, décommentez la ligne ci-dessous ou utilisez la version interactive")
# !python EXO_PRIME_ORCHESTRATOR.py
